# How many cells are identified?
This notebook contains code to check the results of the Identification pileline.

In [51]:
import pickle
import pandas as pd

## Prepare data

In [52]:
# load dictionary
with open('data_RO1/dict_all_cells_v2.pkl', 'rb') as f:
    name_to_CID_dict = pickle.load(f)
    f.close()
    
# add empty dictionary entry for cells without an etl or htl
name_to_CID_dict['none'] = 0

# load dataframe
with open('data_RO1/df_all_ctls.pkl', 'rb') as f:
    df_all_ctls = pickle.load(f)
    
# ETLs: transform entries where multiple materials are separated by a ";" into lists
for index, _ in df_all_ctls.iterrows():
    i = 0
    while i < len(df_all_ctls.at[index, "etl"]):
        if ";" in df_all_ctls.at[index, "etl"][i]:
            elements = df_all_ctls.at[index, "etl"][i].split(";")
            df_all_ctls.at[index, "etl"] = df_all_ctls.at[index, "etl"][:i] + elements + df_all_ctls.at[index, "etl"][i+1:]
        i += 1
        
# HTLs: transform entries where multiple materials are separated by a ";" into lists
for index, _ in df_all_ctls.iterrows():
    i = 0
    while i < len(df_all_ctls.at[index, "htl"]):
        if ";" in df_all_ctls.at[index, "htl"][i]:
            elements = df_all_ctls.at[index, "htl"][i].split(";")
            df_all_ctls.at[index, "htl"] = df_all_ctls.at[index, "htl"][:i] + elements + df_all_ctls.at[index, "htl"][i+1:]
        i += 1

## Combine dictionary info with data

In [53]:
# Create a DataFrame from the dictionary
df_new = pd.DataFrame(list(name_to_CID_dict.items()), columns=['Material', 'CID'])

# Create the 'etl' and 'htl' columns
df_new['etl'] = df_new['Material'].apply(lambda x: df_all_ctls[df_all_ctls['etl'].apply(lambda y: x in y)].index.tolist())
df_new['htl'] = df_new['Material'].apply(lambda x: df_all_ctls[df_all_ctls['htl'].apply(lambda y: x in y)].index.tolist())

In [116]:
# Count how many ETLs are identified
etl_identified = []
etl_count = 0

for index, row in df_all_ctls.iterrows():
    num_etl_materials = len(row['etl'])
    current_num = 0
    for material in row['etl']:
        material = str.strip(material)
        if pd.isna(df_new.loc[df_new['Material'] == material, 'CID'].values[0]):
            break # not identified
        else:
            current_num = current_num + 1 # another element identified
    if current_num == num_etl_materials:
        etl_count = etl_count + 1
        etl_identified.append(True)
    else:
        etl_identified.append(False)

# Count how many HTLs are identified
htl_count = 0
htl_identified = []

for index, row in df_all_ctls.iterrows():
    num_etl_materials = len(row['htl'])
    current_num = 0
    for material in row['htl']:
        material = str.strip(material)
        if pd.isna(df_new.loc[df_new['Material'] == material, 'CID'].values[0]):
            break
        else:
            current_num = current_num + 1
    if current_num == num_etl_materials:
        htl_count = htl_count + 1
        htl_identified.append(True)
    else:
        htl_identified.append(False)
        
# add identification to materials dataframe
df_all_ctls['etls_identified'] = etl_identified
df_all_ctls['htls_identified'] = htl_identified

# create a column stating whether both CTLs were identified
df_all_ctls['both_identified'] = df_all_ctls['etls_identified'] & df_all_ctls['htls_identified']

In [117]:
# ETL: resulting count of identified entries
print(f"{etl_count} entries have an identified ETL.")
print(f"Of these, {len(df_new.at[2449, 'etl'])} entries use no ETL material.")

# HTL: resulting count of identified entries
print(f"{htl_count} entries have an identified HTL.")
print(f"Of these, {len(df_new.at[2449, 'htl'])} entries use no HTL material.")

# Both: resulting count of identified entries
print(f"Both CTLs are identified for {df_all_ctls['both_identified'].sum()} cells.")
# how many cells have no CTL at all? (no HTL, no ETL)
print(f"Of these, {len(list(set(df_new.at[2449, 'htl']) & set(df_new.at[2449, 'etl'])))} cells have neither ETL nor HTL.")

41090 entries have an identified ETL.
Of these, 281 entries use no ETL material.
39647 entries have an identified HTL.
Of these, 2626 entries use no HTL material.
Both CTLs are identified for 37849 cells.
Of these, 31 cells have neither ETL nor HTL.


In [118]:
# save into pickles folder
with open('data_RO1/df_all_ctls_identified_v2.pkl', 'wb') as f:
    pickle.dump(df_all_ctls, f)

In [119]:
# save as excel
df_all_ctls.to_excel('excel_tables/all_cells_identification_info.xlsx', sheet_name='All_cells')

In [120]:
for entry in name_to_CID_dict:
    if name_to_CID_dict[entry]:
        print(f"{entry}:{name_to_CID_dict[entry]}")

TiO2-c:26042
bis-C60:53384373
C60MC12:53384414
Polyacrylonitrile:7855
EPA:446284
MgO-c:14792
CT2:9547904
X25:60210955
Boron subphthalocyanine chloride:11826144
IT-M:145710360
LiF:224478
V1056:2761560
PTT:5462314
ME1:10250490
tert-CuBuPc:71311255
ZrO2:62395
PVP:6917
TaTm:11113597
V1102:2735154
Q198:95377
Cu:23978
Cu2O:10313194
IDIC:132575341
IEICO-4F:139034750
M109:2734487
Au-nw:23985
BaSnO3-nw:13981640
CuP:16214778
Polystyrene:7501
Ba(OH)2:6093286
TOPO:65577
Co3O4:11651651
SQ2:863054
N719:71310680
DTBT:396294
JY6:165619050
YK1:155883041
CuSCN:11029823
rGO:TiO2-nanofibrse:26042
PCBM-60:53384373
TZ3:165416455
CdS:16750131
NiO:938
DTAB:14249
SY2:42601398
SM:23951
IPH:996
Sn2O3-qd:88989
Propylene glycol:1030
B3:5476868
EA:6341
Ni:935
rGO:166001319
ZIF-8:15245636
FDT:5288206
Q219:19309
TFB:118810322
X36:66553046
TT1:18288125
NaCl:5234
PANI:136859416
Au-np:23985
APPA:12901
SubPc:11826144
DFH:69031
18-crown-6 ether:28557
N2,N2,N7,N7-tetrakis(4-methoxyphenyl)spiro[fluorene-9,9'-xanthene]-2,7-d

## Extra: How often were identified materials used?
This question is less relevant and just for the interested.
How often does an identified material appear in the data?

In [121]:
sum_etl = 0
sum_htl = 0

for index, material in enumerate(df_new['Material']):
    if not pd.isna(df_new[df_new['Material'] == material]['CID'].values[0]):
        current_etl_sum = len(df_new[df_new['Material'] == material]['etl'].values[0])
        current_htl_sum = len(df_new[df_new['Material'] == material]['htl'].values[0])
        print(f"Etls with {material}: {current_etl_sum}")
        print(f"Htls with {material}: {current_htl_sum}")
        print(f"{index+1}/2534")
        sum_etl = sum_etl + current_etl_sum
        sum_htl = sum_htl + current_htl_sum

Etls with TiO2-c: 22019
Htls with TiO2-c: 0
1/2534
Etls with bis-C60: 308
Htls with bis-C60: 9
4/2534
Etls with C60MC12: 1
Htls with C60MC12: 0
9/2534
Etls with Polyacrylonitrile: 0
Htls with Polyacrylonitrile: 1
11/2534
Etls with EPA: 1
Htls with EPA: 1
16/2534
Etls with MgO-c: 18
Htls with MgO-c: 0
17/2534
Etls with CT2: 0
Htls with CT2: 1
18/2534
Etls with X25: 0
Htls with X25: 6
20/2534
Etls with Boron subphthalocyanine chloride: 4
Htls with Boron subphthalocyanine chloride: 0
23/2534
Etls with IT-M: 2
Htls with IT-M: 0
27/2534
Etls with LiF: 766
Htls with LiF: 14
28/2534
Etls with V1056: 0
Htls with V1056: 2
30/2534
Etls with PTT: 0
Htls with PTT: 1
34/2534
Etls with ME1: 0
Htls with ME1: 1
36/2534
Etls with tert-CuBuPc: 0
Htls with tert-CuBuPc: 1
38/2534
Etls with ZrO2: 8
Htls with ZrO2: 0
46/2534
Etls with PVP: 6
Htls with PVP: 13
58/2534
Etls with TaTm: 0
Htls with TaTm: 73
61/2534
Etls with V1102: 0
Htls with V1102: 2
62/2534
Etls with Q198: 0
Htls with Q198: 1
63/2534
Etls wi

In [122]:
print(f"ETL identified materials appearances: {sum_etl}")
print(f"HTL identified materials appearances: {sum_htl}")

ETL identified materials appearances: 70253
HTL identified materials appearances: 41781


## Unidentified materials: How many cells pertain to each?

This is interesting to see if there are unidentified materials which are used in many cells. In such cases it might be worth the time to identify them manually.

In [123]:
# Construct dataframe of unidentified materials and their frequency
unidentified_materials = []
no_cells = []

for index, row in df_new.iterrows():
    if pd.isna(row['CID']):
        unidentified_materials.append(row['Material'])
        no_cells.append(len(row['etl']) + len(row['htl']))

df_unid = pd.DataFrame({'Material': unidentified_materials, 'Cell_amount': no_cells})

In [124]:
# save into excel file
df_unid.sort_values(by = 'Cell_amount', ascending=False).to_excel('excel_tables/unidentified.xlsx', sheet_name='Unidentified')

In [125]:
df_unid.sort_values(by = 'Cell_amount', ascending=False).head(40)

,Material,Cell_amount
1344,PEIE,173
530,NiMgLiO,161
1090,P3CT-Na,136
826,PEDOT,60
447,P3CT-N,57
500,PEAI,50
34,P3CT,41
791,TIPD,36
752,NiMgLiO-c,33
707,P3CT-K,30


# Create Excel Sheet for manual evaluation

In [126]:
ref_list_etl = []
ref_list_htl = []

for rowindex, row in df_new.iterrows():
    #etl
    minilist = []
    for index, cell_index in enumerate(row['etl']):
        minilist.append(df_all_ctls['ref'].iloc[cell_index])
    ref_list_etl.append(minilist)
    
    # htl
    minilist = []
    for index, cell_index in enumerate(row['htl']):
        minilist.append(df_all_ctls['ref'].iloc[cell_index])
    ref_list_htl.append(minilist)


df_new['etl_refs'] = ref_list_etl
df_new['htl_refs'] = ref_list_htl

In [141]:
evaluation_df = df_new.drop(columns=['etl',
                                     'htl'])

evaluation_df = evaluation_df.dropna(subset=['CID'])
evaluation_df['CID'] = evaluation_df['CID'].astype(int)

In [146]:
# draw 100 at random 
all_samples = evaluation_df.sample(800, random_state=42)

# split into evaluation batches of 10
samples = [all_samples.iloc[10*i:10*i+10] for i in range(10)]

In [153]:
with pd.ExcelWriter("excel_tables/human_evaluation.xlsx") as writer:

    for index, sample in enumerate(samples):
        sample.to_excel(writer, sheet_name=f"Sample {index+1}", index=False)